In [ ]:
import re
import unicodedata
import PyPDF2
import os

def clean_academic_text(text):
    """
    Clean academic text by removing unwanted patterns and normalizing the content.
    
    Parameters:
    text (str): Raw text from academic paper
    
    Returns:
    str: Cleaned text ready for summarization
    """
    # Remove paper header information
    text = re.sub(r'Jurnal.*?DOI:.*?\d{4}', '', text, flags=re.DOTALL)
    
    # Remove page numbers and headers
    text = re.sub(r'\d+\s*Jurnal.*?\d{4}.*?hlm\.\d+-\d+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+\.\S+', '', text)
    
    # Remove references to tables and figures
    text = re.sub(r'(Tabel|Gambar)\s+\d+\..*?\n', '', text)
    
    # Remove citations
    text = re.sub(r'\([^)]*\d{4}[^)]*\)', '', text)
    
    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    
    # Remove square brackets with numbers
    text = re.sub(r'\[\d+\]', '', text)
    
    # Remove non-standard quotation marks and replace with standard ones
    text = re.sub(r'[""'']', '"', text)
    
    # Remove special characters except periods, commas, and standard punctuation
    text = re.sub(r'[^\w\s.,!?()-]', ' ', text)
    
    # Normalize unicode characters
    text = unicodedata.normalize('NFKC', text)
    
    # Remove multiple periods
    text = re.sub(r'\.{2,}', '.', text)
    
    # Remove spaces before punctuation
    text = re.sub(r'\s+([.,!?])', r'\1', text)
    
    # Remove multiple spaces again (cleanup)
    text = re.sub(r'\s+', ' ', text)
    
    # Strip extra whitespace
    text = text.strip()
    
    return text

def extract_main_content(text):
    """
    Extract the main content from the academic paper, removing metadata and references.
    
    Parameters:
    text (str): Raw text from academic paper
    
    Returns:
    str: Main content of the paper
    """
    # Extract text between abstract and references/conclusion
    main_text = ''
    
    # Find the start (after abstract)
    abstract_match = re.search(r'Abstract.*?\n\n', text, re.IGNORECASE | re.DOTALL)
    if abstract_match:
        start_idx = abstract_match.end()
    else:
        # If no abstract found, try to find introduction
        intro_match = re.search(r'1\.\s*PENDAHULUAN', text)
        start_idx = intro_match.start() if intro_match else 0
    
    # Find the end (before references)
    end_markers = ['DAFTAR PUSTAKA', 'REFERENCES', 'KESIMPULAN', '5\.\s*KESIMPULAN']
    end_idx = len(text)
    
    for marker in end_markers:
        match = re.search(marker, text)
        if match:
            end_idx = min(end_idx, match.start())
    
    # Extract the main content
    main_text = text[start_idx:end_idx]
    
    return main_text

def prepare_text_for_summarization(text):
    """
    Prepare text for summarization by cleaning and extracting main content.
    
    Parameters:
    text (str): Raw text from academic paper
    
    Returns:
    str: Cleaned and prepared text ready for summarization
    """
    # Extract main content first
    main_content = extract_main_content(text)
    
    # Clean the extracted content
    cleaned_text = clean_academic_text(main_content)
    
    return cleaned_text
    
   

In [4]:
    def import_pdf(file_path):
        """
        Import text from a single PDF file.
        
        Parameters:
        file_path (str): Path to the PDF file
        
        Returns:
        str: Extracted text from PDF
        """
        
        text = ''
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text

    def import_multiple_pdfs(directory_path):
        """
        Import text from multiple PDF files in a directory.
        
        Parameters:
        directory_path (str): Path to directory containing PDF files
        
        Returns:
        dict: Dictionary with filenames as keys and extracted text as values
        """
        
        pdf_texts = {}
        for filename in os.listdir(directory_path):
            if filename.endswith('.pdf'):
                file_path = os.path.join(directory_path, filename)
                pdf_texts[filename] = import_pdf(file_path)
        return pdf_texts

In [12]:
# Specify the file path of the PDF
pdf_file = "datasets/ID+8098.pdf"

# Import and process the PDF
pdf_text = import_pdf(pdf_file)

# Clean and prepare the document
cleaned_text = prepare_text_for_summarization(pdf_text)
print("Length of cleaned text:", len(cleaned_text))
print("-" * 50)
print("Cleaned text:")
print(cleaned_text)

Length of cleaned text: 30959
--------------------------------------------------
Cleaned text:
1. PENDAHULUAN Bahasa isyarat merupakan hal yang sangat penting bagi suatu kelompok masyarakat, yaitu masyarakat bisu atau tuli. Untuk masyarakat yang bisu atau tuli, bahasa isyarat adalah metode terpenting untuk berkomunikasi. Tanpa adanya bahasa isyarat, akan sulit bagi mereka yang bisu atau tuli untuk dapat menyatakan maksud atau pikiran mereka. Untuk dapat berkomunik asi dengan masyarakat bisu atau tuli, orang yang tidak bisu atau tuli memerlukan bahasa isyarat tersebut untuk dapat mengerti maksud atau pikiran mereka yang bisu atau tuli. Setiap orang harus memiliki kemampuan menggunakan bahasa isyarat, agar dapat berko munikasi dengan mereka yang bisu atau tuli. Bahasa isyarat diekspresikan menggunakan tangan, lengan, serta wajah dan dimengerti menggunakan mata. Pembuatan sistem klasifikasi bahasa isyarat sebelumnya pernah dilakukan oleh yang membuat asisten virtual untuk bahasa isyarat m

In [11]:
# Process multiple documents
all_cleaned_texts = {}

for filename, text in pdf_text.items():
    # Clean and prepare each document
    cleaned_text = prepare_text_for_summarization(text)
    all_cleaned_texts[filename] = cleaned_text
    # print(f"\nDocument: {filename}")
    # print("Length of cleaned text:", len(cleaned_text))
    # print("-" * 50)
    # print("Cleaned text:")
    # print(cleaned_text)
    

print(all_cleaned_texts)

{'ID+8098.pdf': '1. PENDAHULUAN Bahasa isyarat merupakan hal yang sangat penting bagi suatu kelompok masyarakat, yaitu masyarakat bisu atau tuli. Untuk masyarakat yang bisu atau tuli, bahasa isyarat adalah metode terpenting untuk berkomunikasi. Tanpa adanya bahasa isyarat, akan sulit bagi mereka yang bisu atau tuli untuk dapat menyatakan maksud atau pikiran mereka. Untuk dapat berkomunik asi dengan masyarakat bisu atau tuli, orang yang tidak bisu atau tuli memerlukan bahasa isyarat tersebut untuk dapat mengerti maksud atau pikiran mereka yang bisu atau tuli. Setiap orang harus memiliki kemampuan menggunakan bahasa isyarat, agar dapat berko munikasi dengan mereka yang bisu atau tuli. Bahasa isyarat diekspresikan menggunakan tangan, lengan, serta wajah dan dimengerti menggunakan mata. Pembuatan sistem klasifikasi bahasa isyarat sebelumnya pernah dilakukan oleh yang membuat asisten virtual untuk bahasa isyarat menggunakan deep learning dan tensorflow. Kesimpulan dari penelitian ini adalah

In [13]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
     --------------------------------------- 0.0/12.2 MB 991.0 kB/s eta 0:00:13
     ---------------------------------------- 0.1/12.2 MB 1.7 MB/s eta 0:00:08
      --------------------------------------- 0.3/12.2 MB 2.2 MB/s eta 0:00:06
     - -------------------------------------- 0.4/12.2 MB 2.4 MB/s eta 0:00:05
     - -------------------------------------- 0.5/12.2 MB 2.3 MB/s eta 0:00:06
     - -------------------------------------- 0.6/12.2 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.6/12.2 MB 2.0 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/12.2 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.2 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/12.2 MB 1.9 MB/s eta 0:00:07
     --- ------------------------------------ 0.9/12.2 MB 1.8 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.2 MB 

In [14]:
print(len(all_cleaned_texts))
print(len(cleaned_text))

5
30959


In [15]:
import spacy
import spacy.lang.id.stop_words as STOP_WORDS
from string import punctuation

In [17]:
stopwords = list(STOP_WORDS.STOP_WORDS)


['diminta',
 'nantinya',
 'sana',
 'caranya',
 'buat',
 'termasuk',
 'sesekali',
 'dilihat',
 'bertanya-tanya',
 'diibaratkannya',
 'sekadar',
 'semaunya',
 'dibuatnya',
 'apakah',
 'tertuju',
 'jika',
 'kita',
 'selama',
 'ungkapnya',
 'kira-kira',
 'menanyakan',
 'lah',
 'beberapa',
 'tahun',
 'perlunya',
 'daripada',
 'kenapa',
 'sampai-sampai',
 'diketahuinya',
 'ucapnya',
 'bung',
 'tanpa',
 'sekalian',
 'tutur',
 'memperkirakan',
 'dituturkannya',
 'ketika',
 'ditunjuknya',
 'sela',
 'memperbuat',
 'terlebih',
 'masalah',
 'menjadi',
 'berbagai',
 'diakhirinya',
 'membuat',
 'diantaranya',
 'dikarenakan',
 'berkenaan',
 'anda',
 'adalah',
 'ujar',
 'kalian',
 'kasus',
 'semula',
 'waktunya',
 'bakalan',
 'bermula',
 'berapa',
 'betul',
 'namun',
 'berturut',
 'bukan',
 'tegas',
 'gunakan',
 'minta',
 'lanjutnya',
 'awal',
 'mulanya',
 'mengucapkan',
 'sama-sama',
 'adapun',
 'malah',
 'bersiap-siap',
 'sinilah',
 'juga',
 'mengenai',
 'seseorang',
 'bukankah',
 'padahal',
 'sekit

In [18]:
nlp = spacy.load('en_core_web_sm')

In [19]:
doc = nlp(cleaned_text)

In [20]:
tokens = [token.text for token in doc]
print(tokens)

['1', '.', 'PENDAHULUAN', 'Bahasa', 'isyarat', 'merupakan', 'hal', 'yang', 'sangat', 'penting', 'bagi', 'suatu', 'kelompok', 'masyarakat', ',', 'yaitu', 'masyarakat', 'bisu', 'atau', 'tuli', '.', 'Untuk', 'masyarakat', 'yang', 'bisu', 'atau', 'tuli', ',', 'bahasa', 'isyarat', 'adalah', 'metode', 'terpenting', 'untuk', 'berkomunikasi', '.', 'Tanpa', 'adanya', 'bahasa', 'isyarat', ',', 'akan', 'sulit', 'bagi', 'mereka', 'yang', 'bisu', 'atau', 'tuli', 'untuk', 'dapat', 'menyatakan', 'maksud', 'atau', 'pikiran', 'mereka', '.', 'Untuk', 'dapat', 'berkomunik', 'asi', 'dengan', 'masyarakat', 'bisu', 'atau', 'tuli', ',', 'orang', 'yang', 'tidak', 'bisu', 'atau', 'tuli', 'memerlukan', 'bahasa', 'isyarat', 'tersebut', 'untuk', 'dapat', 'mengerti', 'maksud', 'atau', 'pikiran', 'mereka', 'yang', 'bisu', 'atau', 'tuli', '.', 'Setiap', 'orang', 'harus', 'memiliki', 'kemampuan', 'menggunakan', 'bahasa', 'isyarat', ',', 'agar', 'dapat', 'berko', 'munikasi', 'dengan', 'mereka', 'yang', 'bisu', 'atau',

In [21]:
# langkah selanjutnya adalah me remove tanda baca dan stopwords
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

In [23]:
word_frequencies

{'1': 6,
 'PENDAHULUAN': 1,
 'Bahasa': 5,
 'isyarat': 32,
 'kelompok': 1,
 'masyarakat': 4,
 'bisu': 7,
 'tuli': 7,
 'bahasa': 30,
 'metode': 9,
 'terpenting': 1,
 'berkomunikasi': 1,
 'sulit': 1,
 'maksud': 4,
 'pikiran': 2,
 'berkomunik': 1,
 'asi': 1,
 'orang': 2,
 'mengerti': 1,
 'memiliki': 23,
 'kemampuan': 5,
 'berko': 1,
 'munikasi': 1,
 'diekspresikan': 2,
 'tangan': 4,
 'lengan': 2,
 'wajah': 4,
 'dimengerti': 2,
 'mata': 2,
 'Pembuatan': 1,
 'sistem': 10,
 'klasifikasi': 9,
 'asisten': 1,
 'virtual': 1,
 'deep': 3,
 'learning': 17,
 'tensorflow': 1,
 'Kesimpulan': 3,
 'penelitian': 11,
 'mendeteksi': 8,
 'dataset': 36,
 'bentuk': 6,
 'dunia': 1,
 'regional': 1,
 'lokasi': 1,
 'latar': 31,
 'mendukun': 1,
 'g': 1,
 'mengambil': 3,
 'gambar': 12,
 'sembarang': 1,
 'Penelitian': 5,
 'memfokuskan': 1,
 'pembuatan': 1,
 'aplikasinya': 1,
 'akurasi': 30,
 'performa': 5,
 'mendalam': 2,
 'mend': 1,
 'alami': 1,
 'model': 65,
 '-nya': 8,
 'deteksi': 1,
 'citra': 6,
 'Covid': 1,
 '19

In [25]:
max_frequency = max(word_frequencies.values())
max_frequency

65

In [26]:
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

In [27]:
print(word_frequencies)

{'1': 0.09230769230769231, 'PENDAHULUAN': 0.015384615384615385, 'Bahasa': 0.07692307692307693, 'isyarat': 0.49230769230769234, 'kelompok': 0.015384615384615385, 'masyarakat': 0.06153846153846154, 'bisu': 0.1076923076923077, 'tuli': 0.1076923076923077, 'bahasa': 0.46153846153846156, 'metode': 0.13846153846153847, 'terpenting': 0.015384615384615385, 'berkomunikasi': 0.015384615384615385, 'sulit': 0.015384615384615385, 'maksud': 0.06153846153846154, 'pikiran': 0.03076923076923077, 'berkomunik': 0.015384615384615385, 'asi': 0.015384615384615385, 'orang': 0.03076923076923077, 'mengerti': 0.015384615384615385, 'memiliki': 0.35384615384615387, 'kemampuan': 0.07692307692307693, 'berko': 0.015384615384615385, 'munikasi': 0.015384615384615385, 'diekspresikan': 0.03076923076923077, 'tangan': 0.06153846153846154, 'lengan': 0.03076923076923077, 'wajah': 0.06153846153846154, 'dimengerti': 0.03076923076923077, 'mata': 0.03076923076923077, 'Pembuatan': 0.015384615384615385, 'sistem': 0.153846153846153

In [28]:
sentence_tokens = [sent for sent in doc.sents]
print(sentence_tokens)

[1. PENDAHULUAN Bahasa isyarat merupakan hal yang sangat penting bagi suatu kelompok masyarakat, yaitu masyarakat bisu atau tuli., Untuk masyarakat yang bisu atau tuli, bahasa isyarat adalah metode terpenting untuk berkomunikasi., Tanpa adanya bahasa isyarat, akan sulit bagi mereka yang bisu atau tuli untuk dapat menyatakan maksud atau pikiran mereka., Untuk dapat berkomunik asi dengan masyarakat bisu atau tuli, orang yang tidak bisu atau tuli memerlukan bahasa isyarat tersebut untuk dapat mengerti maksud atau pikiran mereka yang bisu atau tuli., Setiap orang harus memiliki kemampuan menggunakan bahasa isyarat, agar dapat berko munikasi dengan mereka yang bisu atau tuli., Bahasa isyarat diekspresikan menggunakan tangan, lengan, serta wajah dan dimengerti menggunakan mata., Pembuatan sistem klasifikasi bahasa isyarat sebelumnya pernah dilakukan oleh yang membuat asisten virtual untuk bahasa isyarat menggunakan deep learning dan tensorflow., Kesimpulan dari penelitian ini adalah untuk da

In [29]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]

In [30]:
sentence_scores

{1. PENDAHULUAN Bahasa isyarat merupakan hal yang sangat penting bagi suatu kelompok masyarakat, yaitu masyarakat bisu atau tuli.: 1.4000000000000001,
 Untuk masyarakat yang bisu atau tuli, bahasa isyarat adalah metode terpenting untuk berkomunikasi.: 1.4,
 Tanpa adanya bahasa isyarat, akan sulit bagi mereka yang bisu atau tuli untuk dapat menyatakan maksud atau pikiran mereka.: 1.2769230769230773,
 Untuk dapat berkomunik asi dengan masyarakat bisu atau tuli, orang yang tidak bisu atau tuli memerlukan bahasa isyarat tersebut untuk dapat mengerti maksud atau pikiran mereka yang bisu atau tuli.: 1.830769230769231,
 Setiap orang harus memiliki kemampuan menggunakan bahasa isyarat, agar dapat berko munikasi dengan mereka yang bisu atau tuli.: 1.6615384615384616,
 Bahasa isyarat diekspresikan menggunakan tangan, lengan, serta wajah dan dimengerti menggunakan mata.: 1.2000000000000002,
 Pembuatan sistem klasifikasi bahasa isyarat sebelumnya pernah dilakukan oleh yang membuat asisten virtual 

In [31]:
from heapq import nlargest

In [33]:
select_length = int(len(sentence_tokens)*0.3)
select_length

63

In [34]:
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)

In [35]:
print(summary)

[Dataset Total Benar Akurasi Latar Belakang Pengujian Putih Hitam Abstrak Latar Belakang Putih Total Benar 22 23 13 Akurasi 85 88 50 Latar Belakang Hitam Total Benar 13 16 6 Akurasi 50 62 23 Latar Belakang Karpet Total Benar 9 12 3 Akurasi 35 46 12 Latar Belakang Pemandangan Total Benar 9 4 3 Akurasi 35 15 12 Gabungan Total Benar 26 26 23 Akurasi 100 100 88, Hasil dari pengujian ditunjukkan pada Model Akurasi Waktu Training Train Valid ation Test ResNet18 99,89 100 54,73 3585s AlexNet 99,93 100 57,39 3350s ResNet50 99,83 100 57,10 4167s Efficient Net B4 99,96 100 78,10 4924s Hasil pengujian pada Tabel 2 menunjukkan bahwa model dengan arsitektur EfficientNet B4 memiliki akurasi testing yang tertinggi sebesar 78,10., Adapun perhitungan yang dilakukan pada lapisan convolutional adalah sebagai berikut H(x,y) b F(x,y) G(x,y) b F(j,k) k ( ) j ( ) G(x j,y k) (1) Keterangan F Filter Lapisan G Input feature map H Output feature map b Bias x Sumbu X pada data dan filter y Sumbu Y pada data dan f

In [36]:
len(summary)

63

In [37]:
final_summary = [word.text for word in summary]

In [38]:
print(final_summary)

['Dataset Total Benar Akurasi Latar Belakang Pengujian Putih Hitam Abstrak Latar Belakang Putih Total Benar 22 23 13 Akurasi 85 88 50 Latar Belakang Hitam Total Benar 13 16 6 Akurasi 50 62 23 Latar Belakang Karpet Total Benar 9 12 3 Akurasi 35 46 12 Latar Belakang Pemandangan Total Benar 9 4 3 Akurasi 35 15 12 Gabungan Total Benar 26 26 23 Akurasi 100 100 88', 'Hasil dari pengujian ditunjukkan pada Model Akurasi Waktu Training Train Valid ation Test ResNet18 99,89 100 54,73 3585s AlexNet 99,93 100 57,39 3350s ResNet50 99,83 100 57,10 4167s Efficient Net B4 99,96 100 78,10 4924s Hasil pengujian pada Tabel 2 menunjukkan bahwa model dengan arsitektur EfficientNet B4 memiliki akurasi testing yang tertinggi sebesar 78,10.', 'Adapun perhitungan yang dilakukan pada lapisan convolutional adalah sebagai berikut H(x,y) b F(x,y) G(x,y) b F(j,k) k ( ) j ( ) G(x j,y k) (1) Keterangan F Filter Lapisan G Input feature map H Output feature map b Bias x Sumbu X pada data dan filter y Sumbu Y pada data 

In [39]:
summary = ' '.join(final_summary)

In [40]:
summary

'Dataset Total Benar Akurasi Latar Belakang Pengujian Putih Hitam Abstrak Latar Belakang Putih Total Benar 22 23 13 Akurasi 85 88 50 Latar Belakang Hitam Total Benar 13 16 6 Akurasi 50 62 23 Latar Belakang Karpet Total Benar 9 12 3 Akurasi 35 46 12 Latar Belakang Pemandangan Total Benar 9 4 3 Akurasi 35 15 12 Gabungan Total Benar 26 26 23 Akurasi 100 100 88 Hasil dari pengujian ditunjukkan pada Model Akurasi Waktu Training Train Valid ation Test ResNet18 99,89 100 54,73 3585s AlexNet 99,93 100 57,39 3350s ResNet50 99,83 100 57,10 4167s Efficient Net B4 99,96 100 78,10 4924s Hasil pengujian pada Tabel 2 menunjukkan bahwa model dengan arsitektur EfficientNet B4 memiliki akurasi testing yang tertinggi sebesar 78,10. Adapun perhitungan yang dilakukan pada lapisan convolutional adalah sebagai berikut H(x,y) b F(x,y) G(x,y) b F(j,k) k ( ) j ( ) G(x j,y k) (1) Keterangan F Filter Lapisan G Input feature map H Output feature map b Bias x Sumbu X pada data dan filter y Sumbu Y pada data dan fil

In [41]:
print("Summary:")
print(summary)

Summary:
Dataset Total Benar Akurasi Latar Belakang Pengujian Putih Hitam Abstrak Latar Belakang Putih Total Benar 22 23 13 Akurasi 85 88 50 Latar Belakang Hitam Total Benar 13 16 6 Akurasi 50 62 23 Latar Belakang Karpet Total Benar 9 12 3 Akurasi 35 46 12 Latar Belakang Pemandangan Total Benar 9 4 3 Akurasi 35 15 12 Gabungan Total Benar 26 26 23 Akurasi 100 100 88 Hasil dari pengujian ditunjukkan pada Model Akurasi Waktu Training Train Valid ation Test ResNet18 99,89 100 54,73 3585s AlexNet 99,93 100 57,39 3350s ResNet50 99,83 100 57,10 4167s Efficient Net B4 99,96 100 78,10 4924s Hasil pengujian pada Tabel 2 menunjukkan bahwa model dengan arsitektur EfficientNet B4 memiliki akurasi testing yang tertinggi sebesar 78,10. Adapun perhitungan yang dilakukan pada lapisan convolutional adalah sebagai berikut H(x,y) b F(x,y) G(x,y) b F(j,k) k ( ) j ( ) G(x j,y k) (1) Keterangan F Filter Lapisan G Input feature map H Output feature map b Bias x Sumbu X pada data dan filter y Sumbu Y pada data

In [42]:
len(summary)

13218